In [4]:
import paho.mqtt.client as mqtt
import time
import ssl # SSL/TLS 사용 위해 import

In [5]:
# --- MQTT 브로커 정보 ---
MQTT_BROKER = "980ce8dfb90a4c1f923f97df872e7302.s1.eu.hivemq.cloud"
MQTT_PORT = 8883  # MQTT over TLS/SSL 포트
MQTT_USERNAME = "poiu0987"
MQTT_PASSWORD = "Qwer1234" # 실제 비밀번호 입력
 
# --- 토픽 설정 ---
SUB_TOPIC = "test/kotlin/phone" # 라즈베리파이가 구독할 토픽 (예시)
PUB_TOPIC = "test/kotlin/pc"  # 라즈베리파이가 발행할 토픽 (예시)

In [7]:
# --- 콜백 함수 정의 ---

# 브로커 연결 성공 시 호출되는 콜백
def on_connect(client, userdata, flags, rc, properties=None):
    if rc == 0:
        print("MQTT 브로커에 성공적으로 연결되었습니다.")
        # 연결 성공 시 토픽 구독
        client.subscribe(SUB_TOPIC)
        print(f"'{SUB_TOPIC}' 토픽 구독 시작")
    else:
        print(f"MQTT 연결 실패 (Code: {rc})")
 
# 메시지 수신 시 호출되는 콜백
def on_message(client, userdata, msg):
    print(f"메시지 수신 - 토픽: {msg.topic}, 메시지: {msg.payload.decode('utf-8')}")
    # 여기서 수신된 메시지에 따라 라즈베리파이 동작 제어 (예: GPIO 제어)
 
# 메시지 발행 완료 시 호출되는 콜백 (QoS 1 또는 2 사용 시)
def on_publish(client, userdata, mid, rc, properties=None):
    print(f"메시지 발행 완료 (MID: {mid})")

In [8]:
# --- MQTT 클라이언트 설정 및 실행 ---
 
# 클라이언트 인스턴스 생성 (ClientID는 자동 생성됨, 필요시 mqtt.Client("myRaspberryPi") 처럼 지정 가능)
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2) # 최신 콜백 API 버전 사용 권장
 
# 콜백 함수 등록
client.on_connect = on_connect
client.on_message = on_message
client.on_publish = on_publish

# 사용자 이름 및 비밀번호 설정
client.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)
 
# TLS/SSL 설정 (포트 8883 사용 시 필수)
# 기본 CA 인증서를 사용하거나, 필요시 특정 인증서 파일 경로 지정 가능
client.tls_set(tls_version=ssl.PROTOCOL_TLSv1_2) # 명시적으로 TLS 버전 지정 권장
 
# 브로커 연결 시도
try:
    client.connect(MQTT_BROKER, MQTT_PORT, 60) # 60초 타임아웃
except Exception as e:
    print(f"MQTT 연결 중 오류 발생: {e}")
    exit() # 연결 실패 시 종료
 
# 네트워크 루프 시작 (백그라운드 스레드에서 실행, 메시지 수신 및 자동 재연결 처리)
client.loop_start()
 
# --- 메시지 발행 (예시) ---
try:
    while True:
        # 10초마다 라즈베리파이 상태 발행 (예시)
        status_message = f"Raspberry Pi is alive at {time.strftime('%Y-%m-%d %H:%M:%S')}"
        result = client.publish(PUB_TOPIC, status_message, qos=1) # QoS 1로 발행 시도
        result.wait_for_publish() # 발행 완료 기다리기 (선택 사항)
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
             print(f"'{PUB_TOPIC}' 토픽으로 메시지 발행 성공: {status_message}")
        else:
            print(f"메시지 발행 실패 (Code: {result.rc})")
 
        time.sleep(10)
 
except KeyboardInterrupt:
    print("스크립트 종료.")
finally:
    client.loop_stop() # 네트워크 루프 정지
    client.disconnect() # 브로커 연결 해제
    print("MQTT 연결 해제됨.")

MQTT 브로커에 성공적으로 연결되었습니다.
'test/kotlin/phone' 토픽 구독 시작
메시지 발행 완료 (MID: 1)
'test/kotlin/pc' 토픽으로 메시지 발행 성공: Raspberry Pi is alive at 2025-05-22 13:33:46
메시지 발행 완료 (MID: 3)
'test/kotlin/pc' 토픽으로 메시지 발행 성공: Raspberry Pi is alive at 2025-05-22 13:33:57
메시지 수신 - 토픽: test/kotlin/phone, 메시지: hello
메시지 수신 - 토픽: test/kotlin/phone, 메시지: hello
메시지 수신 - 토픽: test/kotlin/phone, 메시지: hello
메시지 발행 완료 (MID: 4)
'test/kotlin/pc' 토픽으로 메시지 발행 성공: Raspberry Pi is alive at 2025-05-22 13:34:07
메시지 발행 완료 (MID: 5)
'test/kotlin/pc' 토픽으로 메시지 발행 성공: Raspberry Pi is alive at 2025-05-22 13:34:17
스크립트 종료.
MQTT 연결 해제됨.
